# New York taxis trips

This homework is about New York taxi trips. Here is something from [Todd Schneider](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/):

> The New York City Taxi & Limousine Commission has released a  detailed historical dataset covering over 1 billion individual taxi trips in the city from January 2009 through December 2019. 
Taken as a whole, the detailed trip-level data is more than just a vast list of taxi pickup and drop off coordinates: it's a story of a City. 
How bad is the rush hour traffic from Midtown to JFK? 
Where does the Bridge and Tunnel crowd hang out on Saturday nights?
What time do investment bankers get to work? How has Uber changed the landscape for taxis?
The dataset addresses all of these questions and many more.

The NY taxi trips dataset has been plowed by series of distinguished data scientists.
The dataset is available from on Amazon S3 (Amazon's cloud storage service).
The link for each file has the following form:

    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{year}-{month}.csv

There is one CSV file for each NY taxi service (`yellow`, `green`, `fhv`) and each calendar month (replacing `{year}` and `{month}` by the desired ones).
Each file is moderately large, a few gigabytes. 
The full dataset is relatively large if it has to be handled on a laptop (several hundred gigabytes).

You will focus on the `yellow` taxi service and a pair of months, from year 2015 and from year 2018. 
Between those two years, for hire vehicles services have taken off and carved a huge marketshare.

Whatever the framework you use, `CSV` files prove hard to handle. 
After downloading the appropriate files (this takes time, but this is routine), a first step will consist in converting the csv files into a more Spark friendly format such as `parquet`.

Saving into one of those formats require decisions about bucketing, partitioning and so on. Such decisions influence performance. It is your call.
Many people have been working on this dataset, to cite but a few:


- [1 billion trips with a vengeance](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/)
- [1 billion trips with R and SQL ](http://freerangestats.info/blog/2019/12/22/nyc-taxis-sql)
- [1 billion trips with redshift](https://tech.marksblogg.com/billion-nyc-taxi-rides-redshift.html)
- [nyc-taxi](https://github.com/fmaletski/nyc-taxi-map)

Depending on your internet connection, **download the files** corresponding to **"yellow" taxis** for the years 2015 and 2018. Download **at least one month** (the same) for 2015 and 2018, if you can download all of them.

**Hint.** The 12 csv for 2015 are about 23GB in total, but the corresponding parquet file, if you can create it for all 12 months, is only about 3GB.

You **might** need the following stuff in order to work with GPS coordinates and to plot things easily.

In [1]:
!pip install geojson geopandas plotly geopy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install ipyleaflet

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


For this homework **we will let you decide on the tools to use** (expected for Spark) and to **find out information all by yourself** (but don't hesitate to ask questions on the `slack` channel).

# Loading data as parquet files

We want to organize the data on a per year and per service basis. 
We want to end up with one `parquet` file for each year and each taxi service, since parquet is much better than CSV files.

**Hint.** Depending on your internet connection and your laptop, you can use only the "yellow" service and use one month of 2015 and 2018

CSV files can contain corrupted lines. You may have to work in order to perform ETL (Extract-Transform-Load) in order obtain a properly typed data frame.

You are invited to proceed as follows:

1. Try to read the CSV file without imposing a schema. 
1. Inspect the inferred schema. Do you agree with Spark's typing decision?
1. Eventually correct the schema and read again the data
1. Save the data into parquet files
1. In the rest of your work, **you will only use the parquet files you created**, not the csv files (don't forget to choose a partitioning column and a number of partitions when creating the parquet files).

**Hint.** Don't forget to ask `Spark` to use all the memory and ressources from your computer.

**Hint.** Don't foreget that you should specify a partitioning column and a number of partitions when creating the parquet files.

**Hint.** Note that the schemas of the 2015 and 2018 data are different...

**Hint.** When working on this, ask you and answer to the following questions:

1. What is the `StorageLevel` of the dataframe after reading the csv files?
1. What is the number of partitions of the dataframe? 
1. Is it possible to tune this number at loading time? 
1. Why would we want to modify the number of partitions when creating the parquet files?

In [1]:
# import the usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import sys
import timeit
import plotly.graph_objects as go
import plotly.express as px

%matplotlib inline
import seaborn as sns

sns.set_context("notebook", font_scale=1.2)

In [2]:
# spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
import pyspark.sql.functions as fn
from pyspark.sql.catalog import Catalog
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, LongType, BooleanType

In [3]:
# Start the SparkSession
conf = SparkConf().setAppName("Spark SQL Illustrations")
sc = SparkContext(conf=conf)

spark = (SparkSession
    .builder
    .appName("New York taxis trips")
    .getOrCreate()
)

1.Try to read the CSV file without imposing a schema.

In [4]:
import requests
from pathlib import Path

# download the data of 2015
path2015 = Path('yellow_tripdata_2015-07.csv')
if not path2015.exists():
    url = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-07.csv"
    r = requests.get(url)
    with open(os.path.join('./', 'yellow_tripdata_2015-07.csv'), 'wb') as f:
        f.write(r.content)

In [5]:
# download the data of 2018
path2018 = Path('yellow_tripdata_2018-07.csv')
if not path2018.exists():
    url = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-07.csv"
    r = requests.get(url)
    with open(os.path.join('./', 'yellow_tripdata_2018-07.csv'), 'wb') as f:
        f.write(r.content)

In [16]:
# Load data from a csv file
df_sp2015 = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("yellow_tripdata_2015-07.csv")

df_sp2015.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [17]:
df_sp2018 = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("yellow_tripdata_2018-07.csv")

df_sp2018.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



2.Inspect the inferred schema. Do you agree with Spark's typing decision?

Non, les colonnes "tpep_pickup_datetime" et "tpep_dropoff_datetime" doivent être de type date et pas de type string.

3.Eventually correct the schema and read again the data

In [18]:
df_sp2015 = df_sp2015\
            .withColumn("tpep_pickup_datetime",(fn.to_timestamp(col("tpep_pickup_datetime"))))\
            .withColumn("tpep_dropoff_datetime",(fn.to_timestamp(col("tpep_dropoff_datetime"))))

df_sp2015.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [19]:
df_sp2018 = df_sp2018\
            .withColumn("tpep_pickup_datetime",(fn.to_timestamp(col("tpep_pickup_datetime"))))\
            .withColumn("tpep_dropoff_datetime",(fn.to_timestamp(col("tpep_dropoff_datetime"))))

df_sp2018.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



4.Save the data into parquet files

In [20]:
# set the number of partitions
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [21]:
df_sp2015.write.mode('overwrite').partitionBy('VendorID').parquet("yellow_tripdata_2015-07.parquet")
df_sp2015.rdd.getNumPartitions()

14

In [22]:
df_sp2018.write.mode('overwrite').partitionBy('VendorID').parquet("yellow_tripdata_2018-07.parquet")
df_sp2018.rdd.getNumPartitions()

6

In [23]:
df2015 = spark.read.parquet("yellow_tripdata_2015-07.parquet")

df2015.show(10)

+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|VendorID|
+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+
| 2015-07-04 03:16:00|  2015-07-04 03:30:19|              1|         5.12|-73.98783874511719| 40.72852325439453|         1|             

In [24]:
df2018 = spark.read.parquet("yellow_tripdata_2018-07.parquet")

df2018.show(10)

+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|VendorID|
+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+
| 2018-07-16 21:37:23|  2018-07-16 22:24:14|              1|        18.14|         2|                 N|         132|          68|           1|       52.0|  0.0|    0.5|      10.0|        5.76|                  0.3|       68.56|       2|
| 2018-07-15 20:04:19|  2018-07-15 20:14:13|    

In [25]:
# Cleaning the data
def cleanData(df, year, month):
    df = df\
        .withColumn('year', fn.year('tpep_pickup_datetime'))\
        .withColumn('month', fn.month('tpep_pickup_datetime'))\
        .where(col('year')==year)\
        .where(col('month')==month)\
        .where(col('tpep_pickup_datetime')<=col('tpep_dropoff_datetime'))
    return df

df2018 = cleanData(df2018, '2018','7')
df2015 = cleanData(df2015, '2015','7')

# Investigate (at least) one month of data in 2015

From now on, you will be using **the parquet files you created for 2015**.

We shall visualize several features of taxi traffic during one calendar month
in 2015 and the same calendar month in 2018.

**Hint.** In order to build appealing graphics, you may stick to `matplotlib + seaborn`, you can use also
`plotly`, which is used a lot to build interactive graphics, but you can use whatever you want.

The following longitudes and lattitudes encompass Newark and JFK airports, Northern Manhattan and Verazzano bridge.

In [26]:
long_min = -74.10
long_max = -73.70
lat_min = 40.58
lat_max = 40.90

1. Using these boundaries, **filter the 2015 data** (using pickup and dropoff longitude and latitude) and count the number of trips for each value of `passenger_count` and make a plot of that.

Trips with $0$ or larger than $7$ passengers are pretty rare.
We suspect these to be outliers. 
We need to explore these trips further in order order to understand what might be wrong
with them

1. What's special with trips with zero passengers?
1. What's special with trips with more than $6$ passengers?
1. What is the largest distance travelled during this month? Is it the first taxi on the moon?
1. Plot the distribution of the `trip_distance` (using an histogram for instance) during year 2105. Focus on trips with non-zero trip distance and trip distance less than 30 miles.

Let's look at what Spark does for these computations

1. Use the `explain` method or have a look at the [Spark UI](http://localhost:4040/SQL/) to analyze the job. You should be able to assess 
    - Parsed Logical Plan
    - Analyzed Logical Plan
    - Optimized Logical Plan
    - Physical Plan
1. Do the Analyzed Logical Plan and Optimized Logical Plan differ? Spot the differences if any. How would a RDBMS proceed with such a query?
1. How does the physical plan differ from the Optimized Logical Plan? What are the keywords you would not expects in a RDBMS? What is their meaning? 
1. Inspect the stages on [Spark UI](http://localhost:4040/stages/stage). How many *stages* are necessary to complete the Spark job? What are the roles of `HashAggregate` and `Exchange hashpartitioning`?
1. Does the physical plan perform `shuffle` operations? If yes how many?
1. What are tasks with respect to stages (in Spark language)? How many tasks are your stages made of?

Now, compute the following and produce relevant plots:

1. Break down the trip distance distribution for each day of week
1. Count the number of distinct pickup location
1. Compute and display tips and profits as a function of the pickup location

# Investigate one month of trips data in 2015 and 2018

 Consider one month of trips data from `yellow` taxis for each year

1. Filter and cache/persist the result

## Assessing seasonalities and looking at time series

Compute and plot the following time series indexed by day of the week and hour of day:

1. The number of pickups
1. The average fare
1. The average trip duration
1. Plot the average of ongoing trips

## Rides to the airports

In order to find the longitude and lattitude of JFK and Newark airport as well as the longitude and magnitudes 
of Manhattan, you can use a service like [geojson.io](http://geojson.io/).
Plot the following time series, indexed the day of the week and hour of the day

1. Median duration of taxi trip leaving Midtown (Southern Manhattan) headed for JFK Airport
1. Median taxi duration of trip leaving from JFK Airport to Midtown (Southern Manhattan)

## Geographic information

For this, you will need to find tools to display maps and to build choropeth maps.
We let you look and find relevant tools to do this.

1. Build a heatmap where color is a function of
    1. number of `pickups`
    2. number of `dropoffs`
    3. number of `pickups` with dropoff at some airport (JFK, LaGuardia, Newark)
2. Build a choropeth map where color is a function of
    1. number of pickups in the area
    1. ratio of number of payments by card/number of cash payments for pickups in the area
    2. ratio of total fare/trip duration for dropoff in the area
3. Build an interactive chorophet with a slider allowing the user to select an `hour of day` and where the color is a function of
    1. average number of dropoffs in the area during that hour the day
    2. average ratio of tip over total fare amount for pickups in the area at given hour of the day

In [27]:
# 4.1 

# ajouter des colonnes
def set_date(df):
    df = df\
        .withColumn('weekofyear', fn.weekofyear('tpep_pickup_datetime'))\
        .withColumn('dayofweek', fn.date_format(col('tpep_pickup_datetime'), "EEEE"))\
        .withColumn('date', fn.to_date(col('tpep_pickup_datetime'), "MM/dd/yyyy"))\
        .withColumn('hour', fn.hour('tpep_pickup_datetime'))\
        .withColumn('tripDurationInMinutes', fn.round((col('tpep_dropoff_datetime').cast('long') - col('tpep_pickup_datetime').cast('long'))/60, 2))
    return df

# The functions
func_num_pickups = fn.count(col('tpep_pickup_datetime'))
func_average_fare = fn.round(fn.avg(col('fare_amount')), 2)
func_average_duration = fn.round(fn.avg(col('tripDurationInMinutes')), 2)

# The function of time series
def time_series(df, typeTimeSerie, function):
    if (typeTimeSerie == 'day_of_week'):
        argGroupBy1 = 'weekofyear'
        argGroupBy2 = 'dayofweek'
    elif (typeTimeSerie == 'hour_of_day'):
        argGroupBy1 = 'date'
        argGroupBy2 = 'hour'
    else:
        print("type time serie error")
        return
    df_ts = set_date(df)
    df_ts = df_ts\
                .groupBy(argGroupBy1, argGroupBy2)\
                .agg(function.alias('num_time_series'))\
                .orderBy(argGroupBy1, argGroupBy2)
    return df_ts

In [28]:
df2015 = set_date(df2015).cache()
df2015.show()

+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+----+-----+----------+---------+----------+----+---------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|VendorID|year|month|weekofyear|dayofweek|      date|hour|tripDurationInMinutes|
+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------

In [29]:
df2018 = set_date(df2018).cache()
df2018.show()

+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+----+-----+----------+---------+----------+----+---------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|VendorID|year|month|weekofyear|dayofweek|      date|hour|tripDurationInMinutes|
+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+----+-----+----------+---------+----------+----+---------------------+
| 2018-07-16 21:37:23|  2018-07-16 22:24:14|              1|        18.14|  

In [30]:
num_pickups_dw_2015 = time_series(df2015, 'day_of_week', func_num_pickups).cache()
num_pickups_hd_2015 = time_series(df2015, 'hour_of_day', func_num_pickups).cache()
avg_fare_dw_2015 = time_series(df2015, 'day_of_week', func_average_fare).cache()
avg_fare_hd_2015 = time_series(df2015, 'hour_of_day', func_average_fare).cache()
avg_trip_dur_dw_2015 = time_series(df2015, 'day_of_week', func_average_duration).cache()
avg_trip_dur_hd_2015 = time_series(df2015, 'hour_of_day', func_average_duration).cache()

num_pickups_dw_2018 = time_series(df2018, 'day_of_week', func_num_pickups).cache()
num_pickups_hd_2018 = time_series(df2018, 'hour_of_day', func_num_pickups).cache()
avg_fare_dw_2018 = time_series(df2018, 'day_of_week', func_average_fare).cache()
avg_fare_hd_2018 = time_series(df2018, 'hour_of_day', func_average_fare).cache()
avg_trip_dur_dw_2018 = time_series(df2018, 'day_of_week', func_average_duration).cache()
avg_trip_dur_hd_2018 = time_series(df2018, 'hour_of_day', func_average_duration).cache()

In [31]:
# 4.1
# plot
def new_update_layout(fig,my_title,x,y,my_legend_title=None) :
    fig.update_layout(
        title={'text' : my_title,
               'x':0.5,
               'xanchor': 'center'},
        legend_title = my_legend_title,
        xaxis_title = x,
        yaxis_title = y          
    )
    return

def plot_time_series(df, typeTimeSerie, title, xaxis_title, yaxis_title):
    if (typeTimeSerie == 'day_of_week'):
        xaxis = 'weekofyear'
        group = 'dayofweek'
    elif (typeTimeSerie == 'hour_of_day'):
        xaxis = 'date'
        group = 'hour'
    else:
        print("type time serie error")
        return
    yaxis = 'num_time_series'
    plot_pd = df.toPandas()
    fig = px.line(plot_pd, 
                  x=xaxis, 
                  y=yaxis, 
                  color=group, 
                  line_group=group, 
                  hover_name=group)
    new_update_layout(fig, title, x=xaxis_title, y=yaxis_title)
    fig.show()

4.1.1 The number of pickups

In [32]:
plot_time_series(num_pickups_dw_2015, 
                 'day_of_week',
                 "The number of pickups in 2015 by day of week",
                 "the number of week in 2015", 
                 "the number of pickups"
                )

plot_time_series(num_pickups_hd_2015, 
                 'hour_of_day',
                 "The number of pickups in 2015 by hour of day",
                 "date", 
                 "the number of pickups"
                )

plot_time_series(num_pickups_dw_2018, 
                 'day_of_week',
                 "The number of pickups in 2018 by day of week",
                 "the number of week in 2018", 
                 "the number of pickups"
                )

plot_time_series(num_pickups_hd_2018, 
                 'hour_of_day',
                 "The number of pickups in 2018 by hour of day",
                 "date", 
                 "the number of pickups"
                )

4.1.2 The average fare

In [33]:
plot_time_series(avg_fare_dw_2015, 
                 'day_of_week',
                 "The average fare in 2015 by day of week",
                 "the number of week in 2015", 
                 "the average fare"
                )

plot_time_series(avg_fare_hd_2015, 
                 'hour_of_day',
                 "The average fare in 2015 by hour of day",
                 "date", 
                 "the average fare"
                )

plot_time_series(avg_fare_dw_2018, 
                 'day_of_week',
                 "The average fare in 2018 by day of week",
                 "the number of week in 2018", 
                 "the average fare"
                )

plot_time_series(avg_fare_hd_2018, 
                 'hour_of_day',
                 "The average fare in 2018 by hour of day",
                 "date", 
                 "the average fare"
                )

4.1.3 The average trip duration

In [34]:
plot_time_series(avg_trip_dur_dw_2015, 
                 'day_of_week',
                 "The average trip duration in 2015 by day of week",
                 "the number of week in 2015", 
                 "the average trip duration"
                )

plot_time_series(avg_trip_dur_hd_2015, 
                 'hour_of_day',
                 "The average trip duration in 2015 by hour of day",
                 "date", 
                 "the average trip duration"
                )

plot_time_series(avg_trip_dur_dw_2018, 
                 'day_of_week',
                 "The average trip duration in 2018 by day of week",
                 "the number of week in 2018", 
                 "the average trip duration"
                )

plot_time_series(avg_trip_dur_hd_2018, 
                 'hour_of_day',
                 "The average trip duration in 2018 by hour of day",
                 "date", 
                 "the average trip duration"
                )

In [14]:
!pip install shapely

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [35]:
# Southern Manhattan
South_Manhattan = {"type": "Polygon",
                   "coordinates": [[
                                [-73.99755477905273,40.77313187935118],
                                [-74.01420593261717,40.751418432997454],
                                [-74.02210235595702,40.70549780669077],
                                [-74.01540756225586,40.69847032728747],
                                [-73.97592544555664,40.70992213555912],
                                [-73.96905899047852,40.73177921058233],
                                [-73.9712905883789,40.74127439314326],
                                [-73.95824432373047,40.758700379161006],
                                [-73.99703979492188,40.774041868909734],
                                [-73.99755477905273,40.77313187935118]]]
                  }
South_Manhattan_id = [12,88,87,261,13,209,231,45,232,148,144,211,125,114,158,249,113,79,4,224,107,234,90,68,246,186,164,170,137,233,229,162,161,230,48,50,163,100]

# JFK Airport
JFK = {"type": "Polygon",
       "coordinates": [[
                       [-73.78778457641602,40.666251560504264],
                       [-73.8226318359375,40.66475414828327],
                       [-73.82649421691895,40.6546620153016],
                       [-73.8226318359375,40.64469860601899],
                       [-73.78469467163085,40.61864344909241],
                       [-73.77053260803221,40.61949040153005],
                       [-73.74701499938965,40.634994248282894],
                       [-73.74881744384766,40.64632671574881],
                       [-73.78778457641602,40.666251560504264]]]
      }
JFK_id = [132]

# LaGuardia Airport
LaGuardia = {"type": "Polygon",
             "coordinates": [[
                                [-73.88777732849121,40.76656658538413],
                                [-73.8758897781372,40.77144186567577],
                                [-73.86966705322266,40.7709543537425],
                                [-73.86297225952148,40.76630656038832],
                                [-73.85541915893555,40.76737915693862],
                                [-73.85404586791992,40.77313187935118],
                                [-73.87056827545166,40.787040358887566],
                                [-73.89009475708008,40.77842914365316],
                                [-73.88777732849121,40.76656658538413]]]
                 }
LaGuardia_id = [138]

# Newark Airport
Newark = {"type": "Polygon",
          "coordinates": [[
                          [-74.15797233581543,40.708295578231315],
                          [-74.18097496032715,40.70823051511181],
                          [-74.19479370117188,40.684738575525],
                          [-74.19118881225586,40.67523532779746],
                          [-74.17745590209961,40.66911608150882],
                          [-74.15119171142578,40.70634365699408],
                          [-74.15797233581543,40.708295578231315]]]
         }

Newark_id = [1]

In [36]:
# vérifier si un point est dans un polygone
import shapely.geometry
def in_area_Midtown(longitude, latitude):
    point = shapely.geometry.Point(longitude, latitude)
    poly_shape = shapely.geometry.asShape(South_Manhattan)
    return poly_shape.intersects(point)

def in_area_JFK(longitude, latitude):
    point = shapely.geometry.Point(longitude, latitude)
    poly_shape = shapely.geometry.asShape(JFK)
    return poly_shape.intersects(point)

udf_in_area_Midtown = udf(lambda x,y : in_area_Midtown(x,y), BooleanType())
udf_in_area_JFK = udf(lambda x,y : in_area_JFK(x,y), BooleanType())

# ajouter des colonnes si la location est dans un domaine donné
def add_area_2015(df):
    df = df\
        .withColumn('pickup_in_South_Manhattan', udf_in_area_Midtown(col('pickup_longitude'), col('pickup_latitude')))\
        .withColumn('dropoff_in_South_Manhattan', udf_in_area_Midtown(col('dropoff_longitude'), col('dropoff_latitude')))\
        .withColumn('pickup_in_JFK', udf_in_area_JFK(col('pickup_longitude'), col('pickup_latitude')))\
        .withColumn('dropoff_in_JFK', udf_in_area_JFK(col('dropoff_longitude'), col('dropoff_latitude')))
    return df

def in_id_Midtown(ID):
    return ID in South_Manhattan_id

def in_id_JFK(ID):
    return ID in JFK_id
    
udf_in_id_Midtown = udf(lambda x : in_id_Midtown(x), BooleanType())
udf_in_id_JFK = udf(lambda x : in_id_JFK(x), BooleanType())

def add_area_2018(df):
    df = df\
        .withColumn('pickup_in_South_Manhattan_id',udf_in_id_Midtown(col('PULocationID')))\
        .withColumn('dropoff_in_South_Manhattan_id',udf_in_id_Midtown(col('DOLocationID')))\
        .withColumn('pickup_in_JFK_id',udf_in_id_JFK(col('PULocationID')))\
        .withColumn('dropoff_in_JFK_id',udf_in_id_JFK(col('DOLocationID')))
    return df

In [37]:
df2015 = add_area_2015(df2015)
df2015.show()

+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+----+-----+----------+---------+----------+----+---------------------+-------------------------+--------------------------+-------------+--------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|VendorID|year|month|weekofyear|dayofweek|      date|hour|tripDurationInMinutes|pickup_in_South_Manhattan|dropoff_in_South_Manhattan|pickup_in_JFK|dropoff_in_JFK|
+--------------------+---------------------+---------------+-------------+------------------+------------------+--------

In [38]:
df2018 = add_area_2018(df2018).cache()
df2018.show()

+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+----+-----+----------+---------+----------+----+---------------------+----------------------------+-----------------------------+----------------+-----------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|VendorID|year|month|weekofyear|dayofweek|      date|hour|tripDurationInMinutes|pickup_in_South_Manhattan_id|dropoff_in_South_Manhattan_id|pickup_in_JFK_id|dropoff_in_JFK_id|
+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---

In [39]:
def MedianDuration(year, typeTimeSerie, pickup, dropoff):
    if (year == 2018):
        df_med = df2018
    elif (year == 2015):
        df_med = df2015
    else:
        print("dataframe error")
        return
    if (typeTimeSerie == 'day_of_week'):
        argGroupBy1 = 'weekofyear'
        argGroupBy2 = 'dayofweek'
    elif (typeTimeSerie == 'hour_of_day'):
        argGroupBy1 = 'date'
        argGroupBy2 = 'hour'
    else:
        print("type time serie error")
        return
    pickupCol = 'pickup_in_' + pickup
    dropoffCol = 'dropoff_in_' + dropoff
    df_med = df_med.where(col(pickupCol)).where(col(dropoffCol))
    med = fn.expr('percentile_approx(tripDurationInMinutes, 0.5)')
    df_med = df_med\
            .groupBy(argGroupBy1, argGroupBy2)\
            .agg(med.alias('num_time_series'))\
            .orderBy(argGroupBy1, argGroupBy2)
    return df_med

In [40]:
Midtown_to_JFK_dw_2018 = MedianDuration(2018, 'day_of_week', 'South_Manhattan_id', 'JFK_id').cache()
Midtown_to_JFK_hd_2018 = MedianDuration(2018, 'hour_of_day', 'South_Manhattan_id', 'JFK_id').cache()

In [ ]:
# problème
Midtown_to_JFK_dw_2015 = MedianDuration(2015, 'day_of_week', 'South_Manhattan', 'JFK').cache()
Midtown_to_JFK_dw_2015.show()

4.2.1 Median duration of taxi trip leaving Midtown (Southern Manhattan) headed for JFK Airport

In [41]:
plot_time_series(Midtown_to_JFK_dw_2018, 
                 'day_of_week',
                 "Median duration of taxi trip leaving Midtown headed for JFK Airport in 2018 by day of week",
                 "the number of week in 2018", 
                 "the median trip duration"
                )

plot_time_series(Midtown_to_JFK_hd_2018, 
                 'hour_of_day',
                 "Median duration of taxi trip leaving Midtown headed for JFK Airport in 2018 by hour of day",
                 "date", 
                 "the median trip duration"
                )

4.2.2 Median taxi duration of trip leaving from JFK Airport to Midtown (Southern Manhattan)

In [40]:
JFK_to_Midtown_dw_2018 = MedianDuration(2018, 'day_of_week', 'JFK_id', 'South_Manhattan_id').cache()
JFK_to_Midtown_hd_2018 = MedianDuration(2018, 'hour_of_day', 'JFK_id', 'South_Manhattan_id').cache()

In [41]:
plot_time_series(JFK_to_Midtown_dw_2018, 
                 'day_of_week',
                 "Median taxi duration of trip leaving from JFK Airport to Midtown in 2018 by day of week",
                 "the number of week in 2018", 
                 "the median trip duration"
                )

plot_time_series(JFK_to_Midtown_hd_2018, 
                 'hour_of_day',
                 "Median taxi duration of trip leaving from JFK Airport to Midtown in 2018 by hour of day",
                 "date", 
                 "the median trip duration"
                )